# ETL Project 
## Shalyn Lavoie | Moriah Taylor 
### 22 May 2021

In [1]:
#import packages
import pandas as pd
from sqlalchemy import create_engine

### Load Census Data

In [4]:
#load census data
census_file = "./Resources/acs2017_county_data.csv"
census = pd.read_csv(census_file)
census.head()

,CountyId,State,County,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,1001,Alabama,Autauga County,55036,26899,28137,2.7,75.4,18.9,0.3,...,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,1003,Alabama,Baldwin County,203360,99527,103833,4.4,83.1,9.5,0.8,...,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
2,1005,Alabama,Barbour County,26201,13976,12225,4.2,45.7,47.8,0.2,...,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
3,1007,Alabama,Bibb County,22580,12251,10329,2.4,74.6,22.0,0.4,...,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
4,1009,Alabama,Blount County,57667,28490,29177,9.0,87.4,1.5,0.3,...,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9


### Clean Census Data

In [7]:
#select CountyId, State, County, TotalPop, White, Income, IncomePerCap, Employed
census_reduced = census[["CountyId", "State", "County", "TotalPop", "White", "Income", "IncomePerCap", "Employed"]].copy()
census_reduced.head()

,CountyId,State,County,TotalPop,White,Income,IncomePerCap,Employed
0,1001,Alabama,Autauga County,55036,75.4,55317,27824,24112
1,1003,Alabama,Baldwin County,203360,83.1,52562,29364,89527
2,1005,Alabama,Barbour County,26201,45.7,33368,17561,8878
3,1007,Alabama,Bibb County,22580,74.6,43404,20911,8171
4,1009,Alabama,Blount County,57667,87.4,47412,22021,21380


### Load Census Data

In [22]:
#load broadband data
broadband_file = "./Resources/broadband.csv"
broadband = pd.read_csv(broadband_file)
broadband.head()

,ST,COUNTY ID,COUNTY NAME,BROADBAND AVAILABILITY PER FCC,BROADBAND USAGE
0,AL,1001,Autauga County,0.81,0.28
1,AL,1003,Baldwin County,0.88,0.30
2,AL,1005,Barbour County,0.59,0.18
3,AL,1007,Bibb County,0.29,0.07
4,AL,1009,Blount County,0.69,0.09


### Clean Census Data

In [23]:
#reduce to columns of interest
broadband_reduced = broadband[["COUNTY ID", "BROADBAND USAGE"]].copy()
broadband_reduced.head()

,COUNTY ID,BROADBAND USAGE
0,1001,0.28
1,1003,0.30
2,1005,0.18
3,1007,0.07
4,1009,0.09


In [24]:
#rename columns
broadband_reduced = broadband_reduced.rename(columns={"COUNTY ID": "CountyId", 
                                                      "BROADBAND USAGE": "Broadband_Usage"})
broadband_reduced.head()

,CountyId,Broadband_Usage
0,1001,0.28
1,1003,0.30
2,1005,0.18
3,1007,0.07
4,1009,0.09


### Merge Data

In [25]:
#merge data
merged_data = census_reduced.merge(broadband_reduced, how="left", on="CountyId")
#make all col names lowercase
merged_data.columns = merged_data.columns.str.lower()
merged_data.head()

,countyid,state,county,totalpop,white,income,incomepercap,employed,broadband_usage
0,1001,Alabama,Autauga County,55036,75.4,55317,27824,24112,0.28
1,1003,Alabama,Baldwin County,203360,83.1,52562,29364,89527,0.30
2,1005,Alabama,Barbour County,26201,45.7,33368,17561,8878,0.18
3,1007,Alabama,Bibb County,22580,74.6,43404,20911,8171,0.07
4,1009,Alabama,Blount County,57667,87.4,47412,22021,21380,0.09


### Put Data in 3rd Normal Form

In [29]:
states = merged_data[["countyid", "state"]].copy()
states.head()

,countyid,state
0,1001,Alabama
1,1003,Alabama
2,1005,Alabama
3,1007,Alabama
4,1009,Alabama


In [30]:
county_names = merged_data[["countyid", "county"]].copy()
county_names.head()

,countyid,county
0,1001,Autauga County
1,1003,Baldwin County
2,1005,Barbour County
3,1007,Bibb County
4,1009,Blount County


In [31]:
counties = merged_data.drop(columns=["state", "county"]).copy()
counties.head()

,countyid,totalpop,white,income,incomepercap,employed,broadband_usage
0,1001,55036,75.4,55317,27824,24112,0.28
1,1003,203360,83.1,52562,29364,89527,0.30
2,1005,26201,45.7,33368,17561,8878,0.18
3,1007,22580,74.6,43404,20911,8171,0.07
4,1009,57667,87.4,47412,22021,21380,0.09
